<a href="https://colab.research.google.com/github/menwcode/Data-Science/blob/main/pyspark_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [5]:
#list all the files
!ls -l

total 214784
drwxr-xr-x  1 root root      4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000      4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root 219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz


total 214788
drwxr-xr-x  1 root root      4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000      4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root 219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz


In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [7]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-11-14 03:06:14--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [              <=>   ]   1.59G  3.27MB/s    in 8m 11s  

2020-11-14 03:14:27 (3.31 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1704091917]



In [12]:
!ls -l

total 1878944
-rw-r--r--  1 root root 1704091917 Nov 13 11:58 reported-crimes.csv
drwxr-xr-x  1 root root       4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root  219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz


In [13]:
#move the rows.csv?accessType=DOWNLOAD
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv


mv: cannot stat 'rows.csv?accessType=DOWNLOAD': No such file or directory


In [14]:
!ls -l

total 1878944
-rw-r--r--  1 root root 1704091917 Nov 13 11:58 reported-crimes.csv
drwxr-xr-x  1 root root       4096 Nov  6 17:30 sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 28 08:10 spark-3.0.1-bin-hadoop2.7
-rw-r--r--  1 root root  219929956 Aug 28 09:25 spark-3.0.1-bin-hadoop2.7.tgz


In [15]:
#Laod data in pyspark
from pyspark.sql.functions import to_timestamp, col, lit
df = spark.read.csv('reported-crimes.csv', header = True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
df.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11| 

In [16]:
##List all the columns
df.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [17]:
#Length of the all the fields
len(df.columns)

22

In [18]:
#Count method to get the all the records
df.count()

6753615

In [19]:
#A duplicate of Pandas shape
print((df.count()), (len(df.columns)))

6753615 22


In [20]:
#List the Schema
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [21]:
#show only 2 columns values
df.select('Case Number', 'Arrest').show(5)

+-----------+------+
|Case Number|Arrest|
+-----------+------+
|   JA366925| false|
|   JB147188| false|
|   JB147595| false|
|   JB147230| false|
|   JB147599| false|
+-----------+------+
only showing top 5 rows



In [22]:
#Statistical measure of each columns of the dataframe
df.describe().show()

+-------+------------------+------------------+--------------+------------------+-----------------+---------------+--------------------+-------+--------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+-------------------+--------------------+--------------------+
|summary|                ID|       Case Number|         Block|              IUCR|     Primary Type|    Description|Location Description| Arrest|Domestic|              Beat|          District|              Ward|   Community Area|          FBI Code|      X Coordinate|     Y Coordinate|             Year|          Updated On|           Latitude|           Longitude|            Location|
+-------+------------------+------------------+--------------+------------------+-----------------+---------------+--------------------+-------+--------+------------------+------------------+------------------+-----------------+

In [ ]:
#Adding a new column
#withcolumn function of Spark, lets you add a new column  sexual_crime by checking "Primary Type"
#df.withColumn("sexual_crime")


In [ ]:
from pyspark.sql.types import StringType, DoubleType

Filtering data


In [ ]:
df.filter(df["Primary Type"] == "CRIM SEXUAL ASSAULT").show()

In [ ]:
#Show all the crimes related to Sexual assulat and Predatory
df.filter((df["Primary Type"] == "CRIM SEXUAL ASSAULT") & (df["Description"] == "PREDATORY")).show()


Show Distinct Values

In [ ]:
#Show distinct values of with Truncate set to False
df.select("Primary Type").distinct().show(truncate = False)

In [ ]:
#Grouping Data
df.groupBy('Primary Type').count().show(50, False)

Put order in above output

In [ ]:
df.groupBy("Primary Type").count().orderBy('count', ascending=False).show(50, False)

In [ ]:
df.groupBy("Block").count().orderBy('count', ascending=False).show(50, False)

In [ ]:
#Show all the Arrest by the crime type
df.select("Arrest").distinct().show()

In [ ]:
#What is the arrest percentage
df.filter(df["Arrest"] == "true").count()/df.select("Arrest").count()

0.27754484080007524

# Show the top location of Crime

In [ ]:
#Top crime location in the city
df.groupBy("Location Description").count().orderBy('count', ascending=False).show(50, False)

+---------------------------------+-------+
|Location Description             |count  |
+---------------------------------+-------+
|STREET                           |1770578|
|RESIDENCE                        |1144978|
|APARTMENT                        |698338 |
|SIDEWALK                         |665552 |
|OTHER                            |256793 |
|PARKING LOT/GARAGE(NON.RESID.)   |193760 |
|ALLEY                            |150911 |
|SCHOOL, PUBLIC, BUILDING         |142316 |
|RESIDENCE-GARAGE                 |131605 |
|SMALL RETAIL STORE               |119270 |
|RESIDENCE PORCH/HALLWAY          |117902 |
|VEHICLE NON-COMMERCIAL           |108435 |
|RESTAURANT                       |105608 |
|GROCERY FOOD STORE               |87301  |
|DEPARTMENT STORE                 |83663  |
|GAS STATION                      |71985  |
|RESIDENTIAL YARD (FRONT/BACK)    |69449  |
|CHA PARKING LOT/GROUNDS          |55452  |
|PARK PROPERTY                    |52340  |
|COMMERCIAL / BUSINESS OFFICE   

## String Functions
Display the primary type column in lower and upper characters and the first 4 characters



In [ ]:
from pyspark.sql.functions import lower, upper, substring
df.select(lower(df["Primary Type"]), upper(df["Primary Type"]), substring(df["Primary Type"], 1, 4)).show(5, truncate=False)

+-------------------+-------------------+-----------------------------+
|lower(Primary Type)|upper(Primary Type)|substring(Primary Type, 1, 4)|
+-------------------+-------------------+-----------------------------+
|deceptive practice |DECEPTIVE PRACTICE |DECE                         |
|crim sexual assault|CRIM SEXUAL ASSAULT|CRIM                         |
|burglary           |BURGLARY           |BURG                         |
|theft              |THEFT              |THEF                         |
|crim sexual assault|CRIM SEXUAL ASSAULT|CRIM                         |
+-------------------+-------------------+-----------------------------+
only showing top 5 rows



# Show the oldest date and most recent dates

In [ ]:
from pyspark.sql.functions import  min, max
df.select(min(df["Date"]), max(col('Date'))).show()

+-------------------+-------------------+
|          min(Date)|          max(Date)|
+-------------------+-------------------+
|2001-01-01 00:00:00|2018-11-11 00:00:00|
+-------------------+-------------------+



# What is 3 days earlier than the older date and 3 days later than the most recent dates

In [ ]:
from pyspark.sql.functions import date_add, date_sub
df.select(date_sub(min(df["Date"])), 3)

TypeError: ignored

In [ ]:
#
df.groupBy('Primary Type').max().show(5, False)

# UDF (User Defined Functions): there is a performance impact

In [ ]:
#Traditional Python function
from pyspark.sql.functions import  udf

def price_range(brand):
  if brand in ['Samsung','Apple']:
    return 'High Price'
  elif brand == "MI":
    return "mid Price"
  else:
    return "low price"

In [ ]:
#Now you could call
brand_udf = udf()